# Calculating Team Standings

In [ ]:
%maven org.dflib:dflib-jupyter:1.0.0-M20
%maven org.postgresql:postgresql:42.7.2

In [ ]:
import org.dflib.jupyter.*;
import org.dflib.*;
import org.dflib.csv.*;
import org.dflib.jdbc.*;

import static org.dflib.Exp.*;

DFLibJupyter.init(getKernelInstance());

In [ ]:
var df1 = Csv.load("../data/games.csv");

DFLibJupyter.setMaxDisplayRows(8);
df1

In [ ]:
df1.getColumn("game_state").valueCounts()

In [ ]:
var df2 = df1.rows($col("game_state").in("OFF", "FINAL"))
    .cols("away_team", "home_team", "away_score", "home_score", "overtime")
    .select();
df2

In [ ]:
var df3a = df2.cols("team", "score", "opposing_score", "overtime")
    .select(
        $col("away_team"),
        $col("away_score"),
        $col("home_score"),
        $col("overtime")
    );

var df3h = df2.cols("team", "score", "opposing_score", "overtime")
    .select(
        $col("home_team"),
        $col("home_score"),
        $col("away_score"),
        $col("overtime")
    );

var df3 = df3a.vConcat(df3h);
df3

In [ ]:
var df4 = df3
    .cols("overtime", "goal_diff").map(
        $str("overtime").mapVal(o -> !o.isBlank()),
        $str("score").castAsInt().sub($str("opposing_score").castAsInt())
    );
    
df4;

In [ ]:
var df5 = df4
    .cols("points").map(
        ifExp($int("goal_diff").ge(0), $val(2), ifExp($bool("overtime"), $val(1), $val(0)))
    );

df5

## Aggregate Standings

In [ ]:
var df6 = df5.group("team")
    .agg(
        $col("team").first(),
        $int("points").sum().as("points"),
        $int("goal_diff").sum().as("goal_diff"),
        count().as("games_played")
    )
    .sort($int("points").desc(), $int("games_played").asc(), $int("goal_diff").desc())
    .cols("rank").map(rowNum());

DFLibJupyter.setMaxDisplayRows(32);

df6

In [ ]:
var connector = Jdbc.connector("jdbc:postgresql://localhost:15432/")
    .userName("postgres")
    .password("test")
    .build();

connector

In [ ]:
connector.tableSaver("standings").mergeByColumns("team").save(df6)